In [37]:
with open('/Users/savery/Documents/day one journal backup/Journal.txt', 'r') as f:
    entries = split_entries(f.read())
    write_entries(entries)

In [36]:
from datetime import datetime
from typing import Dict, Optional
from string import ascii_lowercase

DATE_HEADER_MARKER = '\tDate:\t'
DATE_FORMAT = '%B %d, %Y at %H:%M:%S %Z'
ADDITIONAL_HEADERS = ['location', 'weather']

def extract_entry_metadata(entry: str) -> Optional[Dict[str, Optional[str]]]:
    try:
        lines = entry.split('\n')
        date_str = lines[0]
        entry_date = datetime.strptime(date_str, DATE_FORMAT)

        metadata = {'date': entry_date.strftime('%Y-%m-%d')}

        # Extract additional headers
        for line in lines[1:]:
            if line.startswith('\t'):
                parts = line.split('\t')
                name = parts[1][:-1].lower()
                if name in ADDITIONAL_HEADERS:
                    metadata[name] = parts[2]

        return metadata
    except (ValueError, IndexError):
        return None

def split_entries(contents: str) -> Dict[str, str]:
    entries_dict: Dict[str, str] = {}
    entries = contents.split(DATE_HEADER_MARKER)

    for entry in entries:
        if not entry:
            continue

        metadata = extract_entry_metadata(entry)
        if not metadata:
            continue

        date_str = metadata.pop('date')
        base_key = date_str

        # Handle multiple entries on the same day
        suffix = ''  # Start with an empty suffix
        while True:
            key = f"{base_key}{suffix}"
            if key not in entries_dict:
                break
            if not suffix:
                suffix = 'a'  # Start with 'a' if no suffix is present
            else:
                suffix = chr(ord(suffix) + 1)  # Move to the next alphabet
                if suffix > 'z':  # Handle overflow (after 'z')
                    suffix = 'a' + suffix  # Append 'a' to the beginning

        line_skip_count = len([v for v in metadata.values() if v is not None])
        entry_content = entry.split('\n', line_skip_count + 1)[line_skip_count + 1].strip()

        # Construct the YAML front matter
        yaml_front_matter = "---\n"
        for header, value in metadata.items():
            if value:
                yaml_front_matter += f"{header}: {value}\n"
        yaml_front_matter += "---\n\n"

        # Prepend the YAML front matter to the entry content
        full_entry_content = yaml_front_matter + entry_content

        entries_dict[key] = full_entry_content

    return entries_dict

def write_entries(entries_dict: Dict[str, str]):
    for filename, content in entries_dict.items():
        with open(f'output/{filename}.md', 'w') as f:
            f.write(content)
            